Content Based Filtering

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys

assert sys.version_info >= (3, 7)

In [4]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [5]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

import sys
# 코랩의 경우 나눔 폰트를 설치합니다.
if 'google.colab' in sys.modules:
    !sudo apt-get -qq -y install fonts-nanum
    import matplotlib.font_manager as fm
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
    for fpath in font_files:
        fm.fontManager.addfont(fpath)

# 나눔 폰트를 사용합니다.
import matplotlib

matplotlib.rc('font', family='NanumBarunGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [6]:
!pip install pandas scikit-learn

import pandas as pd
import numpy as np

In [7]:
# CSV 파일 경로 지정
data_path = '/content/drive/MyDrive/0-ml-team-project-pm-data-main/processed_data.csv'
data = pd.read_csv(data_path)

# 데이터 확인
print(data.head())


                                   title   type                     genres  \
0                      The Fifth Element  movie  Action, Adventure, Sci-Fi   
1                      Kill Bill: Vol. 1  movie    Action, Crime, Thriller   
2                                Jarhead  movie      Biography, Drama, War   
3                             Unforgiven  movie             Drama, Western   
4  Eternal Sunshine of the Spotless Mind  movie     Drama, Romance, Sci-Fi   

   releaseYear     imdbId  imdbAverageRating  imdbNumVotes  \
0         1997  tt0119116                7.6        517963   
1         2003  tt0266697                8.2       1224222   
2         2005  tt0418763                7.0        212031   
3         1992  tt0105695                8.2        444769   
4         2004  tt0338013                8.3       1107727   

                                  availableCountries  
0                                         AT, CH, DE  
1  AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, C

In [8]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 불러오기 (예시 데이터프레임)
data = pd.DataFrame({
    'title': ['Memento', 'Monster', 'In the Mood for Love', 'Secret Sunshine', 'Nobody Knows'],
    'genres': ['Mystery, Thriller', 'Biography, Crime','Drama', 'Drama, Romance','Drama' ],
    'imdbAverageRating': [8.4, 7.3, 8.1, 7.5, 8.0],
    'availableCountries' : ['KR', 'KR' , 'KR', 'KR' , 'KR'],
    'releaseYear' : [2000, 2003, 2000, 2007, 2004 ]
})

# 데이터 확인
print(data)

                  title             genres  imdbAverageRating  \
0               Memento  Mystery, Thriller                8.4   
1               Monster   Biography, Crime                7.3   
2  In the Mood for Love              Drama                8.1   
3       Secret Sunshine     Drama, Romance                7.5   
4          Nobody Knows              Drama                8.0   

  availableCountries  releaseYear  
0                 KR         2000  
1                 KR         2003  
2                 KR         2000  
3                 KR         2007  
4                 KR         2004  


TF-IDF 벡터화 하여 유사도 계산

In [9]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['genres'])

# TF-IDF 결과 확인
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)


TF-IDF Matrix Shape: (5, 6)


코사인 유사도 계산

In [10]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 유사도 매트릭스 확인
print("Cosine Similarity Matrix:")
print(cosine_sim)


Cosine Similarity Matrix:
[[1.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.        ]
 [0.         0.         1.         0.55645052 1.        ]
 [0.         0.         0.55645052 1.         0.55645052]
 [0.         0.         1.         0.55645052 1.        ]]


In [11]:
# 영화 제목을 기반으로 유사한 영화 추천
def recommend_movies(title, cosine_sim=cosine_sim, top_n=5):
    # 영화 인덱스 매핑
    indices = pd.Series(data.index, index=data['title'])
    idx = indices[title]

    # 해당 영화와 다른 영화 간의 유사도 점수 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 점수 기준 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 영화 top_n 가져오기 (본인 제외)
    sim_scores = sim_scores[1:top_n+1]
    movie_indices = [i[0] for i in sim_scores]

    # 추천 영화 정보 반환
    return data[['title', 'genres', 'releaseYear', 'availableCountries', 'imdbAverageRating']].iloc[movie_indices]


recommended = recommend_movies('Monster')
print("추천 영화 목록:")
print(recommended)


추천 영화 목록:
                  title             genres  releaseYear availableCountries  \
0               Memento  Mystery, Thriller         2000                 KR   
2  In the Mood for Love              Drama         2000                 KR   
3       Secret Sunshine     Drama, Romance         2007                 KR   
4          Nobody Knows              Drama         2004                 KR   

   imdbAverageRating  
0                8.4  
2                8.1  
3                7.5  
4                8.0  


입력 데이터 5개를 넣어서 Monster와 유사한 데이터 4개를 추천해주는 모델


In [12]:
# 'features' 컬럼 재생성
data['features'] = (
    data['genres'] + ' ' +
    data['releaseYear'].astype(str) + ' ' +
    data['availableCountries']
)

# 전체 데이터셋을 사용한 TF-IDF 벡터화 및 코사인 유사도 계산
tfidf_full = TfidfVectorizer(stop_words='english')
tfidf_matrix_full = tfidf_full.fit_transform(data['features'])

# 전체 데이터셋에서 코사인 유사도 계산
cosine_sim_full = cosine_similarity(tfidf_matrix_full, tfidf_matrix_full)

# 추천 함수 수정
def recommend_movies(title, cosine_sim=cosine_sim_full, top_n=5):
    # 영화 인덱스 매핑
    indices = pd.Series(data.index, index=data['title'])
    idx = indices[title]

    # 해당 영화와 다른 영화 간의 유사도 점수 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 점수 기준 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 영화 top_n 가져오기 (본인 제외)
    sim_scores = sim_scores[1:top_n+1]
    movie_indices = [i[0] for i in sim_scores]

    # 추천 영화 정보 반환
    return data[['title', 'genres', 'releaseYear', 'availableCountries', 'imdbAverageRating']].iloc[movie_indices]

# 예시 영화 'Monster'에 대한 추천
recommended = recommend_movies('Monster')
print("추천 영화 목록:")
print(recommended)


추천 영화 목록:
                  title             genres  releaseYear availableCountries  \
2  In the Mood for Love              Drama         2000                 KR   
4          Nobody Knows              Drama         2004                 KR   
3       Secret Sunshine     Drama, Romance         2007                 KR   
0               Memento  Mystery, Thriller         2000                 KR   

   imdbAverageRating  
2                8.1  
4                8.0  
3                7.5  
0                8.4  


전체 데이터 셋을 불러오는 코드

In [13]:
print(data.columns)

Index(['title', 'genres', 'imdbAverageRating', 'availableCountries',
       'releaseYear', 'features'],
      dtype='object')


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 불러오기
file_path = '/content/drive/MyDrive/0-ml-team-project-pm-data-main/processed_data.csv'  # 업로드한 파일 경로
data = pd.read_csv(file_path)

# 'features' 컬럼 생성 (장르, 개봉년도, 시청가능지역 결합)
data['features'] = (
    data['genres'] + ' ' +
    data['releaseYear'].astype(str) + ' ' +
    data['availableCountries']
)

# TF-IDF 벡터화
tfidf_full = TfidfVectorizer(stop_words='english')
tfidf_matrix_full = tfidf_full.fit_transform(data['features'])

# 코사인 유사도 계산
cosine_sim_full = cosine_similarity(tfidf_matrix_full, tfidf_matrix_full)




In [22]:
# 영화 제목을 기반으로 유사한 영화 추천
def recommend_movies(title, cosine_sim=cosine_sim_full, top_n=5):
    # 영화 제목이 데이터셋에 존재하는지 확인
    if title not in data['title'].values:
        raise ValueError(f"'{title}' is not in the dataset.")

    # 영화 인덱스 매핑
    indices = pd.Series(data.index, index=data['title'])
    idx = indices[title]  # 해당 영화의 인덱스를 찾음

    # 해당 영화와 다른 영화 간의 유사도 점수 가져오기
    sim_scores = list(enumerate(cosine_sim[idx].flatten()))  # flatten()을 사용하여 1차원 배열로 변환

    # 유사도 점수 기준 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 영화 top_n 가져오기 (본인 제외)
    sim_scores = sim_scores[1:]  # 본인 제외 (가장 유사한 영화를 제외하고)

    # 유사한 영화가 충분히 많지 않으면 부족한 만큼 가져오기
    top_n = min(top_n, len(sim_scores))  # 유사도 높은 영화가 부족하면 남은 개수만큼 추천

    # 가장 유사한 영화 top_n개 가져오기
    sim_scores = sim_scores[:top_n]

    # 영화 인덱스만 추출
    movie_indices = [i[0] for i in sim_scores]

    # 유효한 인덱스가 데이터프레임 범위 내에 있는지 확인
    valid_movie_indices = [i for i in movie_indices if i < len(data)]  # 유효한 인덱스만 추출

    # 유효한 영화가 없는 경우 예외 처리
    if not valid_movie_indices:
        raise IndexError("No valid movie indices found.")

    # 추천 영화 정보 반환
    return data[['title', 'genres', 'releaseYear', 'availableCountries', 'imdbAverageRating']].iloc[valid_movie_indices]


recommended = recommend_movies('Monster')
print("추천 영화 목록:")
print(recommended)


추천 영화 목록:
      title                   genres  releaseYear availableCountries  \
98  Monster  Biography, Crime, Drama         2003                 KR   

    imdbAverageRating  
98                7.3  
